# *Contrastive Learning* pipeline

In [1]:
%load_ext autoreload
%autoreload 2
import datasets_handler
import evaluation_metrics
from zeroberto import ZeroBERTo
from sentence_transformers.losses import CosineSimilarityLoss
from setfit import SetFitModel, SetFitTrainer
import gc
import pandas as pd

/Users/alealcoforado/Documents/Projetos/ZeroBERTo/zeroberto/datasets_handler.py:162: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  def splitDocuments(docs: pd.Series()) -> list():
0it [00:00, ?it/s]


In [2]:
which_dataset = 'folhauol' 
max_inferences = 1000

hyp_template = "Esse texto está relacionado a {}."

raw_data, data_col, class_col = datasets_handler.getDataset(which_dataset)
classes_list = list(raw_data[class_col].unique())

In [3]:
zeroshot_method = "dotproduct"  

zeroshot_config = {
    'similarity_model' : 'sentence-transformers/stsb-xlm-r-multilingual',
    'setfit_model' : 'sentence-transformers/stsb-xlm-r-multilingual',# 'ricardo-filho/bert-base-portuguese-cased-nli-assin-2'
    'dataset':which_dataset,
    'class_col':class_col,
    'data_col':data_col,
    'split':"fewshot",
    'labeling_method':zeroshot_method,
    'max_inferences':max_inferences,
    'classes':classes_list,
    'template': hyp_template,
    'random_state':422,
    'top_n': 8,
    'training_examples': 8,
    "batch_size" : 8,
    "num_pairs" : 4,
    "num_epochs" : 1,
    'st_train_epochs': 10,
    'st_train_batch_size': 20,
    # Unfreeze the head and freeze the body -> head-only training
    # 'keep_body_frozen_setfit': True,
    # Unfreeze the head and unfreeze the body -> end-to-end training
    'keep_body_frozen_setfit': False,
}

data_to_label = raw_data.sample(zeroshot_config['max_inferences'],random_state=zeroshot_config['random_state']).sort_index()

model = ZeroBERTo(classes_list=zeroshot_config['classes'],labeling_dataset=data_to_label,
                embeddingModel=zeroshot_config['similarity_model'],config=zeroshot_config)
                  

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [4]:
# setfit_model = SetFitModel.from_pretrained('ricardo-filho/bert-base-portuguese-cased-nli-assin-2')
model.contrastiveModel  = SetFitModel.from_pretrained('ricardo-filho/bert-base-portuguese-cased-nli-assin-2', use_differentiable_head=True,
                                        head_params={"out_features":len(classes_list)})

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [5]:
train_dataframe = datasets_handler.splitDataset(model.labeling_dataset,model.config) ### df_test will not be used

# train_data['class_code'] = train_data['class_code'].astype(int)
# train_data['text'] = train_data['text'].astype(str)
train_data = datasets_handler.buildDatasetDict(train_dataframe)


In [6]:
### build trainer for contrastive learning
model.buildTrainer(train_data)

### train
model.contrastive_train()
gc.collect()

### predict labeling dataset with new classifier
model.getPredictions()
# 5000 sentences --- 4min40s ---sts-xlm-rr
# 5000 sentences ---  ---ricardo-filho/bert-portuguese

setfit_all_metrics = evaluation_metrics.get_metrics(model.y_pred, model.trainer.eval_dataset["class_code"])
print(setfit_all_metrics)

setfit_exec_time  = evaluation_metrics.saveResults(model.config,setfit_all_metrics)
print(model.config)

Applying column mapping to training dataset
***** Running training *****
  Num examples = 720
  Num epochs = 1
  Total optimization steps = 90
  Total train batch size = 8


<class 'dict'>


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/90 [00:00<?, ?it/s]